In [ ]:
Channel analysis impact on revenue: which channel generates more revenue? - Linear regression analysis - Google analytics data

In [ ]:
import pandas as pd

In [ ]:
data_AMC = pd.read_csv(r'Montly revenue 2017-2021 - AMC - Sheet1.csv')

In [ ]:
data_AMC.head()

,Month,Channel group,Sessions,Transactions,Transaction revenue
0,2017-12-01,(Other),102,8,14.99
1,2017-12-01,Direct,663,24,3337.28
2,2017-12-01,Display,8,0,0.00
3,2017-12-01,Email,78,5,1652.00
4,2017-12-01,Organic Search,355,9,1035.88


In [ ]:
len(data_AMC)

In [ ]:
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from matplotlib import pyplot
import numpy as np
import statsmodels.api as sm


In c:\users\crist\appdata\local\programs\python\python38\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In c:\users\crist\appdata\local\programs\python\python38\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In c:\users\crist\appdata\local\programs\python\python38\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In c:\users\crist\appdata\local\programs\python\python38\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed tw

In [ ]:
list(data_AMC.columns)

['Month', 'Channel group', 'Sessions', 'Transactions', 'Transaction revenue']

In [ ]:
y=data_AMC['Transaction revenue']

In [ ]:
dummy_channel = pd.get_dummies(data_AMC['Channel group'])

In [ ]:
#we need to drop one to eliminate multicollinearity
dummy_channel=dummy_channel.drop(['(Other)'], axis=1)

In [ ]:
data_AMC = pd.concat([dummy_channel,data_AMC], axis=1)

In [ ]:
data_AMC.head()

,Direct,Display,Email,Organic Search,Paid Search,Referral,Social,Month,Channel group,Sessions,Transactions,Transaction revenue
0,0,0,0,0,0,0,0,2017-12-01,(Other),102,8,14.99
1,1,0,0,0,0,0,0,2017-12-01,Direct,663,24,3337.28
2,0,1,0,0,0,0,0,2017-12-01,Display,8,0,0.00
3,0,0,1,0,0,0,0,2017-12-01,Email,78,5,1652.00
4,0,0,0,1,0,0,0,2017-12-01,Organic Search,355,9,1035.88


In [ ]:
data_AMC.dtypes

Direct                   uint8
Display                  uint8
Email                    uint8
Organic Search           uint8
Paid Search              uint8
Referral                 uint8
Social                   uint8
Month                   object
Channel group           object
Sessions                 int64
Transactions             int64
Transaction revenue    float64
dtype: object

In [ ]:
#month is used as ID, we drop social to eliminate multicolinearity
X_all = data_AMC[[ 'Direct',
 'Display',
 'Email',
 'Organic Search',
 'Paid Search',
 'Referral',
 'Sessions',
 'Transactions',
 ]]

In [ ]:
X_all = X_all.apply(pd.to_numeric, errors='coerce')

In [ ]:
X_all = sm.add_constant(X_all)

model = sm.OLS(y, X_all).fit()

print_model = model.summary()

print(print_model)

                             OLS Regression Results                            
Dep. Variable:     Transaction revenue   R-squared:                       0.914
Model:                             OLS   Adj. R-squared:                  0.912
Method:                  Least Squares   F-statistic:                     431.0
Date:                 Tue, 20 Jul 2021   Prob (F-statistic):          1.12e-167
Time:                         09:11:47   Log-Likelihood:                -3421.2
No. Observations:                  333   AIC:                             6860.
Df Residuals:                      324   BIC:                             6895.
Df Model:                            8                                         
Covariance Type:             nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const          -5376.0473    858.4

In [ ]:
import math

In [ ]:
organic= math.log(.5441)
display=1
email=math.log(.096)
paidsearch=math.log(.2572)#negative
sessions=math.log(.0955)
transactions=math.log(.2852)#negative

In [ ]:
revenue = -5376.04+(display*4638.59)+(email*22780)+(organic*12380)-(paidsearch*40530)+(sessions*.3218)-(54.6990*transactions)

In [ ]:
revenue = -revenue

In [ ]:
math.log(revenue)

8.787434018586827

# 8.78% expected decrease from last year and we had a 6.63% increase

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calc_vif(X):

    # Calculating VIF
    vif = pd.DataFrame()
    vif["variables"] = X_all.columns
    vif["VIF"] = [variance_inflation_factor(X_all.values, i) for i in range(X_all.shape[1])]

    return(vif)

In [ ]:
#checking for multicolinearity
calc_vif(X_all)

,variables,VIF
0,const,4.860672
1,Direct,4.255197
2,Display,1.438948
3,Email,1.536045
4,Organic Search,7.216831
5,Paid Search,2.458194
6,Referral,1.486668
7,Sessions,7.844887
8,Transactions,4.735382
